In [76]:
using JLD2
using PythonPlot
include("MDPModelFunctions2.jl")

plot_over_time_exp2 (generic function with 1 method)

In [47]:
# experiment 1....

# row is 2 obj vs 4 obj
# col is pre-cue, neutral, retro-cue, neutral

exp1_cowan_k_true_7_yr = [1.16 1.14 1.13 1.02; 2.80 .91 1.30 .96]
exp1_cowan_k_true_adult = [1.98 1.94 1.97 1.95; 3.87 2.66 3.64 2.87]

# experiment 2 - 4 objects
# col is IM cued, IM neutral, VSTM cued, VSTM neutral

exp2_cowan_k_true_7_yr = [1.55 1.16 1.00 0.69]
exp2_cowan_k_true_7_adult = [3.54 2.58 3.58 2.65]


# experiment 3

# Load 3 cued, Load 3 Neutral, Load 6 cued, Load 6 neutral
IM_true_7_yr = [1.44 1.27 1.82 0.88]
Short_VSTM_true_7_yr = [1.14 0.77]
Long_VSTM_true_7_yr = [1.19 0.90]

IM_true_adult = [2.63 2.41 4.80 2.46]
Short_VSTM_true_adult = [2.75 2.38 3.97 2.15]
Long_VSTM_true_adult = [2.76 2.30 3.65 1.75]

1×4 Matrix{Float64}:
 2.76  2.3  3.65  1.75

In [70]:
to_save_folder = "/home/erussek/projects/Memory_Models/shimi_1_run1"

function load_results(epsilon, N_Quanta, NT_per_Second; exp_num = 2, on_cluster = false)
    
    if on_cluster
        to_save_folder = "/home/erussek/projects/Memory_Models/shimi_all_parameter_search"
    else
        to_save_folder = "/Users/erussek/Dropbox/Griffiths_Lab_Stuff/code/Memory_Models/shimi_all_parameter_search/exp$(exp_num)"
    end
    
    local file_name = "N_Quanta_$(N_Quanta)_epsilon_$(epsilon)_NT_per_Second_$(NT_per_Second).jld2"
    
    local full_file_path = joinpath(to_save_folder,file_name)

    res = load(full_file_path)
    print(res)
    
    return res["job_res_$(exp_num)"]
end

function get_model_cowan_k(res; exp_num = 1)
    
    if exp_num == 1
    
        # row is 2 obj vs 4 obj
        # col is pre-cue, neutral, retro-cue, neutral
        
        part_pcorr = res;
        model_cowan_p_corr = [part_pcorr; part_pcorr[2,:]']'
        model_cowan_k = cowan_k(model_cowan_p_corr, 2)
        model_cowan_k[2,:] = cowan_k(model_cowan_p_corr[2,:], 4)
        
    elseif exp_num == 2
        
        model_cowan_k = cowan_k(collect(res)',4)
        
    return model_cowan_k
        
    
end




LoadError: syntax: incomplete: "function" at In[70]:21 requires end

In [49]:
eps_vals = 0:.05:1

# quanta values
q_vals = 2:2:80

# 
NT_vals = [100, 200, 400, 800]

job_eps = []
job_q = []
job_nt = []

for ep in eps_vals
    for q in q_vals
        for nt in NT_vals
            
            push!(job_eps, ep)
            push!(job_q, q)
            push!(job_nt, nt)
            
        end
    end
end

n_jobs_total = length(job_nt)

3360

In [60]:
job_idx = 5
epsilon = job_eps[job_idx]
N_Quanta = job_q[job_idx]
NT_per_Second = job_nt[job_idx]

a = load_results(epsilon, N_Quanta, NT_per_Second; exp_num = 2, on_cluster = false)
#b = get_model_cowan_k(a)

Dict{String, Any}("job_res_2" => (0.5448939664855224, 0.5853871080767746, 0.5453970943912008, 0.5876948677545254))

(0.5448939664855224, 0.5853871080767746, 0.5453970943912008, 0.5876948677545254)

In [69]:
cowan_k(collect(a)',4)

1×4 Matrix{Float64}:
 0.359152  0.683097  0.363177  0.701559

In [77]:
a = sim_exp3(epsilon, N_Quanta, NT_per_Second; return_last_only=false)

([0.5608030159440913 0.5595760533333937 … 0.5588693111651181 0.5597058636900717; 0.53158824667813 0.5311536307913209 … 0.5311453078672993 0.5320556075853093], [0.5606577755593036 0.5616057567459085 … 0.6058272652225665 0.6058272652225665; 0.5299876881675327 0.5295778665343496 … 0.55192459274939 0.55192459274939], [0.5590524291219604 0.5600546022277211 … 0.5599299058648461 0.5592328584126205; 0.5322104696349552 0.5309827819295119 … 0.5309939389695694 0.5302394643163519], [0.5596431409569618 0.5602893861970517 … 0.605497585268602 0.605497585268602; 0.5300502624689075 0.5299600732419304 … 0.5499465130256037 0.5499465130256037], [0.5608346159802613 0.5617741157249635 … 0.5605579904616566 0.5603201557981878; 0.5289455405206883 0.5289381508853921 … 0.5298218686005269 0.529327728338794], [0.5623375222289123 0.5624482291706189 … 0.6101131046241034 0.6101131046241034; 0.5310798726123843 0.5311097156743976 … 0.5555510722429982 0.5555510722429982])

In [75]:
a[1]

2×70 Matrix{Float64}:
 0.559493  0.558569  0.557435  0.557971  …  0.558107  0.558365  0.55774
 0.532329  0.531887  0.530532  0.530232     0.53023   0.529182  0.529305